In [ ]:
!pip install -U nltk

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [2]:
import re
import string
import nltk
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

# 1) (optional) Scrape 100 Job Listings that contain the title "Data Scientist" from indeed.com

At a minimum your final dataframe of job listings should contain
- Job Title
- Job Description

In [3]:
max_jobs = 200
title = '"Data Scientist"'
location = 'United States'

def make_urls(job_title, location, max_jobs):
  num_pages = max_jobs // 10
  print(num_pages)
  
  root = 'http://www.indeed.com/jobs?q='
  
  job_title = job_title.replace(' ', '+')
  location = location.replace(' ', '+')
  print(job_title)
  print(location)
  
  URLs = []
  
  pageNum = 0
  for page in range(num_pages):
    
    if pageNum == 0:
        seedURL = root + job_title + '&l=' + location
    else:
        jobNum = pageNum*10
        jobNum = str(jobNum)
        seedURL = root + job_title + '&l=' + location + '&start=' + jobNum
    pageNum = pageNum + 1
    URLs.append(seedURL)

  return URLs
  
urls = make_urls(title, location, max_jobs)
print(urls)

20
"Data+Scientist"
United+States
['http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=10', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=20', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=30', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=40', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=50', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=60', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=70', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=80', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=90', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=100', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=110', 'http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=120', 'http://w

In [94]:
l = []
root = 'http://www.indeed.com/'
for url in urls:
    print('url', url)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    items = soup.select('a.turnstileLink')
    print(len(items))
    for item in items:
        div = item.find_next('div', attrs={"class": 'result-link-bar'})
        print(div.contents[0].text)
        sponsered = (div.contents[0].text == 'Sponsored')
        if sponsered:
            continue
#         print('not sponsered')
#         print('item', item['href'])
        detail = requests.get(root + item['href'])
        print(root + item['href'])
        dsoup = BeautifulSoup(detail.text, 'html.parser')
        dd = {}
        tdiv = dsoup.select('div.jobsearch-JobComponent')
        dd['title'] = tdiv[0].contents[0].find_next('h3').text
        ddiv = dsoup.select(
            'div.jobsearch-JobComponent-description.icl-u-xs-mt--md')
        if len(ddiv) == 0:
            ddiv = dsoup.select(
                'div.jobsearch-JobMetadataHeader-item.icl-u-xs-mt--xs')
            dd['description'] = ddiv[0].find_next('div').text
        else:
            dd['description'] = ddiv[0].text if len(ddiv[0].select('ul')) == 0 else \
                                ddiv[0].select('ul')[0].text

        l.append(dd)
print('l', l)

url http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States
23
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
7 hours ago
not sponsered
http://www.indeed.com//company/First-Soft-Solutions/jobs/Data-Scientist-508782ab929c4b34?fccid=9301486944daefc9&vjs=3
4 hours ago
not sponsered
http://www.indeed.com//rc/clk?jk=d501d413b04f4258&fccid=2bfe06fc096100f7&vjs=3
15 hours ago
not sponsered
http://www.indeed.com//rc/clk?jk=06950fce05467a19&fccid=1f5c1d5ed6808f8d&vjs=3
8 hours ago
not sponsered
http://www.indeed.com//company/Intelligence-Decision-Systems,-Inc./jobs/Data-Scientist-80c896c44069dcfa?fccid=c0af1ae69e78ee28&vjs=3
9 hours ago
not sponsered
http://www.indeed.com//company/Natural-Systems-Analysts/jobs/Data-Scientist-2f2edb46315cd019?fccid=01791819f628a267&vjs=3
14 hours ago
not sponsered
http://www.indeed.com/

http://www.indeed.com//rc/clk?jk=4e857abb03b48ebd&fccid=804a47dd3ecffda7&vjs=3
4 hours ago
not sponsered
http://www.indeed.com//rc/clk?jk=3ea5471c0b69c662&fccid=2b71572aab734bf8&vjs=3
5 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=f9ccd33974757dec&fccid=913e1b259c8d65e2&vjs=3
1 day ago
not sponsered
http://www.indeed.com//company/Factspan-Analytics/jobs/Data-Scientist-ac82ff899d96322b?fccid=e9cd29d0804449f7&vjs=3
4 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=d97614926c37bcff&fccid=d38bfdd9cfc9455e&vjs=3
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
url http://www.indeed.com/jobs?q="Data+Scientist"&l=United+States&start=60
23
1 day ago
not sponsered
http://www.indeed.com//company/Quantcast/jobs/Senior-Data-Scientist-2fb05e1e2afe58b3?f

http://www.indeed.com//rc/clk?jk=5d6dc74808f153fe&fccid=b2a2b698f491af18&vjs=3
18 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=175f025b6468e3e5&fccid=7136762d065a5ad7&vjs=3
8 days ago
not sponsered
http://www.indeed.com//company/Innovu/jobs/Data-Scientist-dcdab7184e4b3b5f?fccid=aa0bad17ef9c5b44&vjs=3
Samsung 
not sponsered
http://www.indeed.com//rc/clk?jk=602e88acaf1bee41&fccid=da3c7fed78dd1607&vjs=3
30+ days ago
not sponsered
http://www.indeed.com//rc/clk?jk=851d784993c4eb81&fccid=5c80575a010bbc63&vjs=3
8 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=1b3cece85a1d070e&fccid=fc49ba29ac1ff036&vjs=3
3 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=da86ab53e4c03727&fccid=f766f8bfbc3effb7&vjs=3
20 hours ago
not sponsered
http://www.indeed.com//rc/clk?jk=02b1de9165dc3161&fccid=07bc7e159c95f065&vjs=3
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Sponsored
sponsered
Spons

23
4 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=8581f40100d96fb1&fccid=8e4ade64e8feb2ea&vjs=3
3 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=44d406f68d42063c&fccid=b533ab2927d30db7&vjs=3
4 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=75b8e7f65f403215&fccid=756537cd73891813&vjs=3
4 months ago
not sponsered
http://www.indeed.com//rc/clk?jk=7aed7cc38549bce3&fccid=48ecd526e3aa3225&vjs=3
12 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=f292ae32e372ca60&fccid=734cb5a01ee60f80&vjs=3
Hasbro Inc. 
not sponsered
http://www.indeed.com//rc/clk?jk=bcb2e6b971f59048&fccid=81e720819d6faa96&vjs=3
Walmart 
not sponsered
http://www.indeed.com//rc/clk?jk=fd14f9e0558ac006&fccid=822bc5d9a49270ea&vjs=3
4 days ago
not sponsered
http://www.indeed.com//rc/clk?jk=607949fb4802ae5f&fccid=17d397c17f760b21&vjs=3
1 day ago
not sponsered
http://www.indeed.com//rc/clk?jk=5d6dc74808f153fe&fccid=b2a2b698f491af18&vjs=3
30+ days ago
not sponsered
http://www.indeed.com//rc/clk?jk

In [95]:
print(len(l))

200


## 2) Use NLTK to tokenize / clean the listings 

In [96]:
wl = l[0:100]
print(len(wl))

100


# 3) Use Scikit-Learn's CountVectorizer to get word counts for each listing.

In [0]:
##### Your Code Here #####

# 4) Visualize the most common word counts

In [0]:
##### Your Code Here #####

 # 5) Use Scikit-Learn's tfidfVectorizer to get a TF-IDF feature matrix

In [0]:
##### Your Code Here #####

## Stretch Goals

 - Scrape Job Listings for the job title "Data Analyst". How do these differ from Data Scientist Job Listings
 - Try and identify requirements for experience specific technologies that are asked for in the job listings. How are those distributed among the job listings?
 - Use a clustering algorithm to cluster documents by their most important terms. Do the clusters reveal any common themes?
  - **Hint:** K-means might not be the best algorithm for this. Do a little bit of research to see what might be good for this. Also, remember that algorithms that depend on Euclidean distance break down with high dimensional data.